In [1]:
%pip install -q ultralytics kagglehub pyyaml opencv-python efficientnet_pytorch torch torchvision

import os
import shutil
import pathlib
import yaml
import cv2
import random
import time
import json
import platform

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision.datasets import ImageFolder

import kagglehub
from ultralytics import YOLO
from efficientnet_pytorch import EfficientNet

dev = "cuda"

random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

raw_p = kagglehub.dataset_download("rupankarmajumdar/crop-pests-dataset")
raw_p = pathlib.Path(raw_p)

base_p = pathlib.Path("/content") if os.path.isdir("/content") else pathlib.Path.cwd()
data_p = base_p / "datasets" / "cp_yolo_eff"
data_p.parent.mkdir(parents=True, exist_ok=True)
shutil.copytree(raw_p, data_p, dirs_exist_ok=True)

tr_img = "train/images"
va_img = "valid/images"
te_img = "test/images"

cls_list = [
    "ant", "aphid", "beetle", "bollworm", "borer", "bug", "caterpillar",
    "grasshopper", "moth", "slug", "snail", "wasp", "weevil"
]

cfg_p = data_p / "data.yaml"
cfg = {
    "path": str(data_p),
    "train": tr_img,
    "val": va_img,
    "test": te_img,
    "names": {i: c for i, c in enumerate(cls_list)},
}
with open(cfg_p, "w") as f:
    yaml.safe_dump(cfg, f)

runs_p = base_p / "yolo_runs"
runs_p.mkdir(parents=True, exist_ok=True)

def get_metrics(v):
    rd = getattr(v, "results_dict", {}) or {}
    per_class = {}
    try:
        maps = v.box.maps
        names = v.names
        for i, ap in enumerate(maps):
            per_class[names.get(i, str(i))] = float(ap)
    except Exception:
        per_class = {}

    return {
        "mAP50-95(B)": float(rd.get("metrics/mAP50-95(B)", 0.0)),
        "precision(B)": float(rd.get("metrics/precision(B)", 0.0)),
        "recall(B)": float(rd.get("metrics/recall(B)", 0.0)),
        "per_class_mAP50-95": per_class,
    }

yolo_1 = YOLO("yolov8n.pt")

ep1 = 10
imgsz = 640
batch = 8

t1_s = time.time()
yolo_1.train(
    data=str(cfg_p),
    epochs=ep1,
    imgsz=imgsz,
    batch=batch,
    device=0,
    workers=2,
    project=str(runs_p),
    name="cp_s1_mosaic",
    verbose=False,
    mosaic=0.5,
    mixup=0.0,
    copy_paste=0.0,
)
t1_e = time.time()
t1_elapsed = t1_e - t1_s

v1 = yolo_1.val()
m1 = get_metrics(v1)

best_1 = runs_p / "cp_s1_mosaic" / "weights" / "best.pt"

yolo_2 = YOLO(str(best_1))

ep2 = 5

t2_s = time.time()
yolo_2.train(
    data=str(cfg_p),
    epochs=ep2,
    imgsz=imgsz,
    batch=batch,
    device=0,
    workers=2,
    project=str(runs_p),
    name="cp_s2_nomosaic",
    verbose=False,
    mosaic=0.0,
    mixup=0.0,
    copy_paste=0.0,
)
t2_e = time.time()
t2_elapsed = t2_e - t2_s

v2 = yolo_2.val()
m2 = get_metrics(v2)

print("\nStage 1")
print("Overall:", {
    "mAP50-95(B)": m1["mAP50-95(B)"],
    "precision(B)": m1["precision(B)"],
    "recall(B)": m1["recall(B)"],
})
print("Per-class mAP50-95:")
for cls, score in m1["per_class_mAP50-95"].items():
    print(f"  {cls}: {score:.4f}")

print("\nStage 2")
print("Overall:", {
    "mAP50-95(B)": m2["mAP50-95(B)"],
    "precision(B)": m2["precision(B)"],
    "recall(B)": m2["recall(B)"],
})
print("Per-class mAP50-95:")
for cls, score in m2["per_class_mAP50-95"].items():
    print(f"  {cls}: {score:.4f}")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Using Colab cache for faster access to the 'crop-pests-dataset' dataset.
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/cp_yolo_eff/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None,